# Setup Notebook
Common setup for data engineering workflows


In [ ]:
# Import framework to verify it's available
from clinical_data_standards_framework import utils
from clinical_data_standards_framework.config_manager import ConfigManager
from pyspark.sql import functions as F
import os
import time
print("✅ Framework loaded successfully")


In [ ]:
# Get parameters from job
flow_name = dbutils.widgets.get("flow_name")
databricks_run_id = dbutils.widgets.get("databricks_run_id")
databricks_job_id = dbutils.widgets.get("databricks_job_id")
databricks_job_name = dbutils.widgets.get("databricks_job_name")
created_by_principal = dbutils.widgets.get("created_by_principal")
workspace_file_path = dbutils.widgets.get("workspace_file_path")

print(f"Flow: {flow_name}")
print(f"Job ID: {databricks_job_id}")
print(f"Job Name: {databricks_job_name}")
print(f"Run ID: {databricks_run_id}")
print(f"Created By: {created_by_principal}")
print(f"Workspace Path: {workspace_file_path}")

import json

start_time = time.time()

# ============================================================================
# OPTIMIZED: Read from Delta cache instead of parsing YAML (~5 sec vs ~60 sec)
# ============================================================================
# Default catalog/schema for config cache table (will be confirmed from cache)
default_catalog = "aira"
default_bronze_schema = "bronze_md"
cache_table = f"{default_catalog}.{default_bronze_schema}.md_config_cache"

use_cache = False
try:
    # Try to read from Delta cache
    if spark.catalog.tableExists(cache_table):
        config_df = spark.table(cache_table).filter(
            F.col("config_key") == "clinical_data_standards"
        ).select("config_section", "config_json")
        
        cache_count = config_df.count()
        if cache_count > 0:
            # Load all sections from cache
            configs = {row.config_section: json.loads(row.config_json) for row in config_df.collect()}
            
            globals_dict = configs.get("globals", {})
            services_dict = configs.get("services", {})
            versioning_dict = configs.get("versioning", {})
            streaming_dict = configs.get("streaming", {})
            pipelines_dict = configs.get("pipelines", {})
            
            # Get pipeline config for this flow
            pipeline_config = pipelines_dict.get(flow_name) if flow_name and pipelines_dict else None
            
            use_cache = True
            elapsed = time.time() - start_time
            print(f"\n⚡ Config loaded from Delta cache in {elapsed:.2f}s")
        else:
            print(f"\n⚠️ Cache table exists but is empty, falling back to YAML")
    else:
        print(f"\n⚠️ Cache table not found, falling back to YAML")
except Exception as e:
    print(f"\n⚠️ Cache read failed ({e}), falling back to YAML")

# Fallback: Load from YAML if cache not available
if not use_cache:
    config_path = f"{workspace_file_path}/config/clinical_data_standards.yaml"
    config = ConfigManager(config_path)
    
    globals_dict = config.get_globals()
    services_dict = config.get_services()
    versioning_dict = config.get_versioning()
    streaming_dict = config.config.get('streaming', {})
    pipeline_config = config.get_pipeline(flow_name) if flow_name else None
    
    elapsed = time.time() - start_time
    print(f"\n📋 Config loaded from YAML in {elapsed:.2f}s")
    print(f"   💡 Run nb_populate_config_cache.ipynb to enable fast loading")

print(f"\n✅ Config loaded:")
print(f"  - Globals: {list(globals_dict.keys())}")
print(f"  - Services: {list(services_dict.keys())}")
print(f"  - Versioning: {list(versioning_dict.keys()) if versioning_dict else 'Not configured'}")
print(f"  - Streaming: {list(streaming_dict.keys()) if streaming_dict else 'Not configured'}")
print(f"  - Pipeline: {flow_name}")

# Display key globals
print(f"\n📦 Globals:")
print(f"  Catalog: {globals_dict.get('catalog')}")
print(f"  Bronze Schema: {globals_dict.get('bronze_schema')}")
print(f"  Silver Schema: {globals_dict.get('silver_schema')}")
print(f"  Gold Schema: {globals_dict.get('gold_schema')}")

# Set task values for downstream tasks - Job tracking
dbutils.jobs.taskValues.set(key="flow_name", value=flow_name)
dbutils.jobs.taskValues.set(key="databricks_run_id", value=databricks_run_id)
dbutils.jobs.taskValues.set(key="databricks_job_id", value=databricks_job_id)
dbutils.jobs.taskValues.set(key="databricks_job_name", value=databricks_job_name)
dbutils.jobs.taskValues.set(key="created_by_principal", value=created_by_principal)

# Set task values for downstream tasks - Config as JSON strings
dbutils.jobs.taskValues.set(key="globals", value=json.dumps(globals_dict))
dbutils.jobs.taskValues.set(key="services", value=json.dumps(services_dict))
dbutils.jobs.taskValues.set(key="versioning", value=json.dumps(versioning_dict))
dbutils.jobs.taskValues.set(key="streaming_config", value=json.dumps(streaming_dict))
if pipeline_config:
    dbutils.jobs.taskValues.set(key="pipeline_config", value=json.dumps(pipeline_config))

print("\n✅ Task values set for downstream tasks")
print("   Downstream notebooks can access via:")
print("   globals = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='globals'))")
print("   services = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='services'))")
print("   versioning = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='versioning'))")
print("   streaming_config = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='streaming_config'))")
print("   pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey='setup', key='pipeline_config'))")

# ================================
# Handle Path Overrides for File Processor
# ================================
# Read optional workflow parameters (widgets defined in job base_parameters)
# dbutils.widgets.get() only takes 1 argument
try:
    source_root_param = dbutils.widgets.get("source_root")
    source_subdir_param = dbutils.widgets.get("source_subdir")
    target_subdir_param = dbutils.widgets.get("target_subdir")
    
    # Compute source/target paths
    if source_subdir_param:
        # Full path override provided
        source_subdir = source_subdir_param
        target_subdir = target_subdir_param if target_subdir_param else source_subdir_param.replace('/uploads', '/ingested')
        print(f"\n🔧 Using workflow parameter override:")
        print(f"  Source: {source_subdir}")
        print(f"  Target: {target_subdir}")
    else:
        # Compute from root (or use default)
        if source_root_param:
            source_root = source_root_param
        else:
            # Get default from config
            source_root = pipeline_config.get('default_source', 'historical_data') if pipeline_config else 'historical_data'
        
        # Compute: {root}/uploads → {root}/ingested
        source_subdir = f"{source_root}/uploads"
        target_subdir = f"{source_root}/ingested"
        print(f"\n📁 Using config default (root: {source_root}):")
        print(f"  Source: {source_subdir}")
        print(f"  Target: {target_subdir}")
    
    # Also get volume names from config
    source_volume = pipeline_config.get('source_volume', 'clinical_data_standards') if pipeline_config else 'clinical_data_standards'
    target_volume = pipeline_config.get('target_volume', 'clinical_data_standards') if pipeline_config else 'clinical_data_standards'
    
    # Set task values for file processor to use
    dbutils.jobs.taskValues.set(key="source_volume", value=source_volume)
    dbutils.jobs.taskValues.set(key="source_subdir", value=source_subdir)
    dbutils.jobs.taskValues.set(key="target_volume", value=target_volume)
    dbutils.jobs.taskValues.set(key="target_subdir", value=target_subdir)
    
    print(f"\n✅ Path configuration set for file processor")
except Exception:
    # These widgets don't exist for non-file-processor jobs - silently skip
    pass
